In [ ]:
import glob
import pickle
from pathlib import Path

import matplotlib
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import pymeshfix
import skimage
import zarr
from joblib import Parallel, delayed
from matplotlib.colors import LightSource
from skimage.measure import marching_cubes
from skimage.transform import rescale
from tqdm import tqdm
from trimesh import Trimesh
from trimesh.smoothing import filter_taubin

matplotlib.rcParams["pdf.fonttype"] = 42
matplotlib.rcParams["ps.fonttype"] = 42
import matplotlib
import seaborn as sns

In [ ]:
def concatenate_overlapping_arrays(arrays):
    def overlap(arr1, arr2):
        return np.any(np.in1d(arr1, arr2))

    groups = []

    for arr in arrays:
        merged = False
        for group in groups:
            if any(overlap(arr, g) for g in group):
                group.append(arr)
                merged = True
                break
        if not merged:
            groups.append([arr])

    # Concatenate arrays within each group
    result = [np.concatenate(group) for group in groups]

    # Remove duplicates within each result array
    result = [np.unique(r) for r in result]

    return result

In [ ]:
out_path = (
    "/Morphodynamics_of_human_early_brain_organoid_development/tracking/lumen_tracking/"
)

In [ ]:
smootheness = 5
label = f"lumen_masks_smooth_{smootheness}_processed"
channel = "GFP"
matrigel = [1, 2, 3, 4]
no_matrix = [5, 6, 7, 8, 13, 14, 15, 16]
agar = [9, 10, 11, 12]

positions = np.arange(1, 17)
all_measurements = pd.DataFrame()
for position in tqdm(positions):
    try:
        # read all csv from one tp --> concat to one df
        morpho_path = (
            f"{out_path}/morphometrics/Position_{position}_Settings_1_Processed/"
        )

        # read graph --> extract all fusion events
        track_path = f"{out_path}/tracks/Position_{position}_Settings_1_Processed/"
        track_graph = pickle.load(
            open(track_path + f"{channel}_{label}_track_motile_graph.pickle", "rb")
        )
        nx_graph = pickle.load(
            open(track_path + f"{channel}_{label}_track.pickle", "rb")
        )
        adjacency_network = nx.to_scipy_sparse_array(nx_graph)
        # Get all in edges for just fused lumen
        in_nodes = np.unique(
            list(
                nx_graph.in_edges(
                    np.array(nx_graph.nodes)[adjacency_network.sum(0) == 2]
                )
            )
        )
        # Get just fused lumen
        post_fusion_nodes = np.array(nx_graph.nodes)[adjacency_network.sum(0) == 2]
        # Only keep nodes which are about to fuse
        fusion_nodes = in_nodes[~np.isin(in_nodes, post_fusion_nodes)] + 1

        csv_names = glob.glob(morpho_path + "*.csv")
        measurements = pd.DataFrame()
        for csv_table in csv_names:
            path = Path(csv_table)
            one_m = pd.read_csv(csv_table)
            one_m["time_point"] = int(path.name.split("_")[1].split(".")[0])
            measurements = pd.concat([measurements, one_m])

        measurements["will_fuse"] = measurements["label"].isin(fusion_nodes).astype(int)
        measurements["position"] = position

        if position in matrigel:
            measurements["Condition"] = "Matrigel"
        if position in no_matrix:
            measurements["Condition"] = "No Matrix"
        if position in agar:
            measurements["Condition"] = "Agarose"

        all_measurements = pd.concat([all_measurements, measurements])
    except:
        print(f"position {position} did not run")

all_measurements["axis_ratio"] = (
    all_measurements["axis_major_length"] / all_measurements["axis_minor_length"]
)
imp_list = list(all_measurements.iloc[:, 1:20].columns) + [
    "axis_minor_length",
    "axis_major_length",
    "axis_ratio",
]

In [ ]:
level = 4

In [ ]:
# check if all lumen have been quantified
for position in np.unique(all_measurements["position"]):
    print(position)
    all_measurements_pos = all_measurements[all_measurements["position"] == position]
    for t in tqdm(range(125)):
        detection_name = "lumen_masks_smooth_5_processed_detection"
        zarr_path = f"/Morphodynamics_of_human_early_brain_organoid_development/tracking/lumen_tracking/Position_{str(position)}_Settings_1_Processed_registered.zarr"
        zarr_array = zarr.open(zarr_path, mode="r")
        input_movie = zarr_array[channel]
        lumen_mask = input_movie[t]["labels"][detection_name][level][:]
        label_mask = np.unique(lumen_mask[lumen_mask != 0])
        labels = np.array(
            all_measurements_pos[all_measurements_pos["time_point"] == t]["label"]
        )
        if (np.isin(label_mask, labels)).all() == False:
            print(t)
            print(labels)

In [ ]:
for position in np.unique(all_measurements["position"]):
    for t in range(125):
        detection_name = "lumen_masks_smooth_5_processed_detection"
        zarr_path = f"/Morphodynamics_of_human_early_brain_organoid_development/tracking/lumen_tracking/Position_{str(position)}_Settings_1_Processed_registered.zarr"
        zarr_array = zarr.open(zarr_path, mode="r")
        input_movie = zarr_array[channel]
        input_shape = input_movie["0"]["labels"][detection_name]["0"].shape
        lumen_mask = input_movie[t]["labels"][detection_name][level][:]

In [ ]:
import plotly.graph_objects as go

In [ ]:
all_measurements["Day"] = (all_measurements["time_point"] / 24) + 4

In [ ]:
measurements_col = pd.DataFrame()
# Collapse all_measurements to per organoid measurements
measurements_col["Volume"] = all_measurements["area"] * 0.347 * 0.347 * 4 * 2 * 4
pallette = {"Matrigel": "#17ad97", "No Matrix": "#4d4d4d", "Agarose": "#98d9d1"}
measurements_col["Day"] = (all_measurements["time_point"] / 24) + 4
measurements_col["Condition"] = all_measurements["Condition"]
measurements_col["Volume of imaged lumen in $\mu m^3*10^6$"] = (
    measurements_col["Volume"] / 10**6
)
measurements_col = measurements_col[
    measurements_col["Day"].isin(np.arange(4, 9.5, 0.5))
]

In [ ]:
cm = 1 / 2.54  # centimeters in inches

In [ ]:
import matplotlib
import seaborn as sns

matplotlib.rcParams["pdf.fonttype"] = 42
matplotlib.rcParams["ps.fonttype"] = 42
fig, ax = plt.subplots(figsize=(4, 4))
sns.despine(left=False, bottom=False, right=True)
# sns.set(font_scale=0.3)

sns.stripplot(
    data=measurements_col,
    x="Day",
    y=r"Volume of imaged lumen in $\mu m^3*10^6$",
    hue="Condition",
    dodge=True,
    size=4,
    palette=pallette,
)
ax.yaxis.set_tick_params(labelsize=8)
ax.xaxis.set_tick_params(labelsize=8)
plt.legend(loc="upper left", bbox_to_anchor=(1, 1), markerscale=1.0, fontsize=8)
"""
plt.savefig(
    "extended_figures/agar_lumen_size_distribution_over_time.pdf",
    bbox_inches="tight",
)
"""

In [ ]:
measurements_col.to_csv("figures_csv/ext_fig_4_g.csv")

In [ ]:
measurements_col = pd.DataFrame()
# Collapse all_measurements to per organoid measurements
all_measurements["id_un"] = (
    all_measurements["position"].astype(str)
    + "_"
    + all_measurements["time_point"].astype(str)
)
measurements_col["Condition"] = all_measurements.groupby(["id_un"])["Condition"].agg(
    pd.Series.mode
)
measurements_col["Mean major axis length"] = all_measurements.groupby("id_un")[
    "axis_major_length"
].mean()
measurements_col["time_point"] = all_measurements.groupby(["id_un"])["time_point"].agg(
    pd.Series.mode
)
measurements_col["Mean major axis length"] = (
    measurements_col["Mean major axis length"] * 0.347 * 4
)
pallette = {"Matrigel": "#17ad97", "No Matrix": "#4d4d4d", "Agarose": "#98d9d1"}
measurements_col["Day"] = (measurements_col["time_point"] / 24) + 4

import seaborn as sns

sns.set_context("paper", rc={"font.size": 8, "axes.titlesize": 8, "axes.labelsize": 8})
rc = {
    "figure.figsize": (10, 5),
    "axes.facecolor": "white",
    "axes.grid": False,
    "grid.color": ".8",
    "font.size": 5,
}
plt.rcParams.update(rc)
cm = 1 / 2.54  # centimeters in inches

fig, ax = plt.subplots(figsize=(5 * cm, 5 * cm))
sns.despine(left=True, bottom=True, right=True)
sns.lineplot(
    x="Day",
    y="Mean major axis length",
    data=measurements_col,
    hue="Condition",
    palette=pallette,
    ci="sd",
)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
"""
plt.savefig(
    "updated_figures_figure_2/mean_major_axis_length_lumen.pdf", bbox_inches="tight"
)
"""

In [ ]:
measurements_col.to_csv("figures_csv/ext_fig_4_h.csv")

In [ ]:
matrigel = [1, 2, 3, 4]
no_matrix = [5, 6, 7, 8, 13, 14, 15, 16]
agar = [9, 10, 11, 12]

In [ ]:
def extract_lumen_epi_volumes(t, position):
    detection_name = "lumen_masks_smooth_5_processed_detection"
    zarr_path = f"/cluster/project/treutlein/DATA/imaging/viventis/Morphodynamics_of_human_early_brain_organoid_development/tracking/lumen_tracking/Position_{str(position)}_Settings_1_Processed_registered.zarr"
    zarr_array = zarr.open(zarr_path, mode="r")
    input_movie = zarr_array[channel]
    input_shape = input_movie["0"]["labels"][detection_name]["0"].shape
    lumen_mask = input_movie[t]["labels"][detection_name][level][:]
    tissue_mask = input_movie[t]["labels"]["tissue_mask"][level][:]
    tissue_mask = (tissue_mask >= 2).astype(int)
    tissue_mask[lumen_mask > 0] = 2
    one_tp_volumes = pd.DataFrame()
    one_tp_volumes["position"] = [position]
    one_tp_volumes["time_point"] = t
    one_tp_volumes["organoid_volume"] = tissue_mask[tissue_mask > 0].size
    one_tp_volumes["lumen_volume"] = tissue_mask[tissue_mask == 2].size
    if position in matrigel:
        one_tp_volumes["Condition"] = "Matrigel"
    if position in no_matrix:
        one_tp_volumes["Condition"] = "No Matrix"
    if position in agar:
        one_tp_volumes["Condition"] = "Agarose"
    return one_tp_volumes

In [ ]:
volume_df = pd.DataFrame()
level = 1
for position in tqdm(range(1, 17)):
    list_df = Parallel(n_jobs=8, backend="multiprocessing", verbose=5)(
        delayed(extract_lumen_epi_volumes)(t, position) for t in range(125)
    )

    volume_df = pd.concat([volume_df] + list_df)
# Save the df
volume_df.index = np.arange(len(volume_df))
volume_df["Volume Organoid"] = (
    volume_df["organoid_volume"] * 0.347 * 0.347 * 4 * 4 * 2
) / 10**6
volume_df["Volume Lumen"] = (
    volume_df["lumen_volume"] * 0.347 * 0.347 * 4 * 4 * 2
) / 10**6
volume_df["Day"] = 4 + (volume_df["time_point"] / 24)
volume_df["Percent Lumen"] = np.array(volume_df["lumen_volume"]) / np.array(
    volume_df["organoid_volume"]
)
volume_df.to_csv(out_path + "tissue_volumes.csv")
volume_df = pd.read_csv(out_path + "tissue_volumes.csv")
volume_df["Volume Epithelium"] = (
    volume_df["Volume Organoid"] - volume_df["Volume Lumen"]
)

In [ ]:
import seaborn as sns

sns.set_context("paper", rc={"font.size": 8, "axes.titlesize": 8, "axes.labelsize": 8})
rc = {
    "figure.figsize": (10, 5),
    "axes.facecolor": "white",
    "axes.grid": False,
    "grid.color": ".8",
    "font.size": 5,
}
plt.rcParams.update(rc)
cm = 1 / 2.54  # centimeters in inches

fig, ax = plt.subplots(figsize=(5 * cm, 5 * cm))
sns.despine(left=True, bottom=True, right=True)
sns.lineplot(
    x="Day",
    y="Percent Lumen",
    data=volume_df,
    hue="Condition",
    palette=pallette,
    ci="sd",
)


sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
"""
plt.savefig(
    "updated_figures_figure_2/percent_lumen_volume_max_line.pdf", bbox_inches="tight"
)
"""

In [ ]:
volume_df[["position", "Day", "Condition", "Percent Lumen"]].to_csv(
    "figures_csv/fig_2_i_percent_lumen.csv"
)

In [ ]:
import seaborn as sns

sns.set_context("paper", rc={"font.size": 8, "axes.titlesize": 8, "axes.labelsize": 8})
rc = {
    "figure.figsize": (10, 5),
    "axes.facecolor": "white",
    "axes.grid": False,
    "grid.color": ".8",
    "font.size": 5,
}
plt.rcParams.update(rc)
cm = 1 / 2.54  # centimeters in inches

fig, ax = plt.subplots(figsize=(5 * cm, 5 * cm))
sns.despine(left=True, bottom=True, right=True)
sns.lineplot(
    x="Day",
    y="Volume Lumen",
    data=volume_df,
    hue="Condition",
    palette=pallette,
    ci="sd",
)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
"""
plt.savefig("updated_figures_figure_2/lumen_volume.pdf", bbox_inches="tight")
"""

In [ ]:
import seaborn as sns

sns.set_context("paper", rc={"font.size": 8, "axes.titlesize": 8, "axes.labelsize": 8})
rc = {
    "figure.figsize": (10, 5),
    "axes.facecolor": "white",
    "axes.grid": False,
    "grid.color": ".8",
    "font.size": 5,
}
plt.rcParams.update(rc)
cm = 1 / 2.54  # centimeters in inches

fig, ax = plt.subplots(figsize=(5 * cm, 5 * cm))
sns.despine(left=True, bottom=True, right=True)
sns.lineplot(
    x="Day",
    y="Volume Organoid",
    data=volume_df,
    hue="Condition",
    palette=pallette,
    ci="sd",
)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
"""
plt.savefig("updated_figures_figure_2/organoid_volume.pdf", bbox_inches="tight")
"""

In [ ]:
volume_df[["position", "Day", "Condition", "Volume Organoid"]].to_csv(
    "figures_csv/ext_fig_4_e_organoid_volume.csv"
)

In [ ]:
import seaborn as sns

sns.set_context("paper", rc={"font.size": 8, "axes.titlesize": 8, "axes.labelsize": 8})
rc = {
    "figure.figsize": (10, 5),
    "axes.facecolor": "white",
    "axes.grid": False,
    "grid.color": ".8",
    "font.size": 5,
}
plt.rcParams.update(rc)
cm = 1 / 2.54  # centimeters in inches

fig, ax = plt.subplots(figsize=(5 * cm, 5 * cm))
sns.despine(left=True, bottom=True, right=True)
sns.lineplot(
    x="Day",
    y="Volume Epithelium",
    data=volume_df,
    hue="Condition",
    palette=pallette,
    ci="sd",
)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
"""
plt.savefig("updated_figures_figure_2/lumen_epithelium.pdf", bbox_inches="tight")
"""

In [ ]:
volume_df[["position", "Day", "Condition", "Volume Epithelium"]].to_csv(
    "figures_csv/ext_fig_4_f_epithelium_volume.csv"
)

In [ ]:
all_measurements["Mean major axis length"] = (
    all_measurements["axis_major_length"] * 0.347 * 4
)
all_pos = all_measurements[all_measurements["position"] == position]
color_measurement = "Mean major axis length"
n_bins = 10

norm = matplotlib.colors.Normalize(
    vmin=all_measurements[color_measurement].min(),
    vmax=all_measurements[color_measurement].max(),
)


cmap_gray_red = matplotlib.colors.LinearSegmentedColormap.from_list(
    "", ["#4166ab", "#f0e0a1", "#bb4f3c"], N=n_bins
)

import matplotlib.colors as colors
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(8, 1))

# Create the colorbar
cbar = plt.colorbar(
    plt.cm.ScalarMappable(norm=norm, cmap=cmap_gray_red),
    cax=ax,
    orientation="horizontal",
    label=r"Major axis length ($\mu m$)",
)

# Set the number of ticks to match n_bins

plt.tight_layout()
"""
plt.savefig(
    "updated_figures_figure_2/major_axis_length.pdf", format="pdf", bbox_inches="tight"
)
"""
plt.show()

In [ ]:
all_measurements[
    ["position", "Condition", "label", "time_point", "area", "axis_major_length"]
].to_csv("figures_csv/fig_2_h_volume_major_axis_length.csv")

In [ ]:
all_measurements["volume"] = (
    all_measurements["area"] * 2 * 0.347 * 4 * 0.347 * 4
) / 10**6
all_pos = all_measurements[all_measurements["position"] == position]
color_measurement = "volume"
n_bins = 10

norm = matplotlib.colors.Normalize(
    vmin=all_measurements[color_measurement].min(),
    vmax=all_measurements[color_measurement].max(),
)


cmap_gray_red = matplotlib.colors.LinearSegmentedColormap.from_list(
    "", ["#00ffea", "#f0e0a1", "#ffdd00"], N=n_bins
)
import matplotlib.colors as colors
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(8, 1))

# Create the colorbar
cbar = plt.colorbar(
    plt.cm.ScalarMappable(norm=norm, cmap=cmap_gray_red),
    cax=ax,
    orientation="horizontal",
    label=r"Lumen volume ($10^6$ $\mu m^3$)",
)

# Set the number of ticks to match n_bins

plt.tight_layout()
plt.savefig(
    "updated_figures_figure_2/colorbar_volume.pdf", format="pdf", bbox_inches="tight"
)

plt.show()

In [ ]:
t = 100
position = 3
level = 2
time_points = [24, 48, 72, 96]
positions = [3, 12, 13]
conditions = ["Matrigel", "Agarose", "No Matrix"]
for position, condition in zip(positions, conditions):
    for t in time_points:
        all_measurements["Mean major axis length"] = (
            all_measurements["axis_major_length"] * 0.347 * 4
        )
        all_pos = all_measurements[all_measurements["position"] == position]
        color_measurement = "Mean major axis length"
        n_bins = 10

        norm = matplotlib.colors.Normalize(
            vmin=all_measurements[color_measurement].min(),
            vmax=all_measurements[color_measurement].max(),
        )

        cmap_gray_red = matplotlib.colors.LinearSegmentedColormap.from_list(
            "", ["#4166ab", "#f0e0a1", "#bb4f3c"], N=n_bins
        )

        label = f"lumen_masks_smooth_5_processed"
        zarr_path = f"/cluster/project/treutlein/DATA/imaging/viventis/Morphodynamics_of_human_early_brain_organoid_development/tracking/lumen_tracking/Position_{str(position)}_Settings_1_Processed_registered.zarr"
        zarr_array = zarr.open(zarr_path, mode="r")
        input_movie = zarr_array[channel]
        input_shape = input_movie["0"]["labels"][label]["0"].shape
        detection_name = "lumen_masks_smooth_5_processed_detection"

        lumen_mask_label = input_movie[t]["labels"][detection_name][level][:]
        organoid_mask = input_movie[t]["labels"]["tissue_mask"][level][:] == 2
        anniso_factor = 2 / (2 * (2**level) * 0.347)
        lumen_mask_label = rescale(
            lumen_mask_label,
            [anniso_factor, 1, 1],
            order=0,
            anti_aliasing=False,
            preserve_range=True,
        ).astype(np.uint16)

        organoid_mask = rescale(
            organoid_mask,
            [anniso_factor, 1, 1],
            order=0,
            anti_aliasing=False,
            preserve_range=True,
        ).astype(np.uint16)

        # Create surface mask
        fig = plt.figure(figsize=(6, 6))
        fig.subplots_adjust(top=1, bottom=0, left=0, right=1, wspace=0)
        ax = fig.add_subplot(111, projection="3d")
        ax.set_box_aspect([1, 1, 1])
        max_size = (np.array(lumen_mask_label.shape) * 2 * (level**1) * 0.347).max() + 0
        ax.set_xlim3d([0, max_size])
        ax.set_ylim3d([0, max_size])
        ax.set_zlim3d([0, max_size])
        shapes = lumen_mask_label.shape

        ax.view_init(15, 15)
        ax.tick_params(axis="both", pad=5)
        ls = LightSource(azdeg=0, altdeg=-65)
        labels_frame = np.unique(lumen_mask_label)[1:]
        measurements_labels = all_pos[all_pos["label"].isin(labels_frame)]

        for label in tqdm(labels_frame):
            measuremnt_lum = measurements_labels[measurements_labels["label"] == label][
                color_measurement
            ]
            vertices, faces, _, _ = marching_cubes(
                (lumen_mask_label == label).astype(int), 0, step_size=1
            )
            vertices_clean, faces_clean = pymeshfix.clean_from_arrays(vertices, faces)
            vertices_clean = np.round(vertices_clean * 2 * (level**1) * 0.347).astype(
                int
            )

            cell_mesh = Trimesh(vertices=vertices_clean, faces=faces_clean)
            cell_mesh = filter_taubin(cell_mesh, iterations=50)

            ax.plot_trisurf(
                cell_mesh.vertices[:, 0],
                cell_mesh.vertices[:, 1],
                triangles=cell_mesh.faces,
                Z=cell_mesh.vertices[:, 2],
                lightsource=ls,
                alpha=1.0,
                color=cmap_gray_red(norm(measuremnt_lum.iloc[0])),
            )
        plt.axis("off")
        canvas = plt.gca().figure.canvas
        canvas.draw()
        data = np.frombuffer(canvas.tostring_rgb(), dtype=np.uint8)
        image = data.reshape(canvas.get_width_height()[::-1] + (3,))
        plt.close()
        # plt.imshow(image[100:450,150:500])
        # plt.show()
        skimage.io.imsave(
            f"updated_figures_figure_2/3D_colored/3D_lumen_{color_measurement}_{condition}_day_{int((t/24)+4)}.png",
            image[100:450, 150:500],
        )

In [ ]:
t = 100
position = 3
level = 2
time_points = [24, 48, 72, 96]
positions = [3, 12, 13]
conditions = ["Matrigel", "Agarose", "No Matrix"]
for position, condition in zip(positions, conditions):
    for t in time_points:
        all_measurements["volume"] = (
            all_measurements["area"] * 2 * 0.347 * 4 * 0.347 * 4
        )
        all_pos = all_measurements[all_measurements["position"] == position]
        color_measurement = "volume"
        n_bins = 10

        norm = matplotlib.colors.Normalize(
            vmin=all_measurements[color_measurement].min(),
            vmax=all_measurements[color_measurement].max(),
        )

        cmap_gray_red = matplotlib.colors.LinearSegmentedColormap.from_list(
            "", ["#00ffea", "#f0e0a1", "#ffdd00"], N=n_bins
        )
        label = f"lumen_masks_smooth_5_processed"
        zarr_path = f"/cluster/project/treutlein/DATA/imaging/viventis/Morphodynamics_of_human_early_brain_organoid_development/tracking/lumen_tracking/Position_{str(position)}_Settings_1_Processed_registered.zarr"
        zarr_array = zarr.open(zarr_path, mode="r")
        input_movie = zarr_array[channel]
        input_shape = input_movie["0"]["labels"][label]["0"].shape
        detection_name = "lumen_masks_smooth_5_processed_detection"

        lumen_mask_label = input_movie[t]["labels"][detection_name][level][:]
        organoid_mask = input_movie[t]["labels"]["tissue_mask"][level][:] == 2
        anniso_factor = 2 / (2 * (2**level) * 0.347)
        lumen_mask_label = rescale(
            lumen_mask_label,
            [anniso_factor, 1, 1],
            order=0,
            anti_aliasing=False,
            preserve_range=True,
        ).astype(np.uint16)

        organoid_mask = rescale(
            organoid_mask,
            [anniso_factor, 1, 1],
            order=0,
            anti_aliasing=False,
            preserve_range=True,
        ).astype(np.uint16)

        # Create surface mask
        fig = plt.figure(figsize=(6, 6))
        fig.subplots_adjust(top=1, bottom=0, left=0, right=1, wspace=0)
        ax = fig.add_subplot(111, projection="3d")
        ax.set_box_aspect([1, 1, 1])
        max_size = (np.array(lumen_mask_label.shape) * 2 * (level**1) * 0.347).max() + 0
        ax.set_xlim3d([0, max_size])
        ax.set_ylim3d([0, max_size])
        ax.set_zlim3d([0, max_size])
        shapes = lumen_mask_label.shape

        ax.view_init(15, 15)
        ax.tick_params(axis="both", pad=5)
        ls = LightSource(azdeg=0, altdeg=-65)
        labels_frame = np.unique(lumen_mask_label)[1:]
        measurements_labels = all_pos[all_pos["label"].isin(labels_frame)]

        for label in tqdm(labels_frame):
            measuremnt_lum = measurements_labels[measurements_labels["label"] == label][
                color_measurement
            ]
            vertices, faces, _, _ = marching_cubes(
                (lumen_mask_label == label).astype(int), 0, step_size=1
            )
            vertices_clean, faces_clean = pymeshfix.clean_from_arrays(vertices, faces)
            vertices_clean = np.round(vertices_clean * 2 * (level**1) * 0.347).astype(
                int
            )

            cell_mesh = Trimesh(vertices=vertices_clean, faces=faces_clean)
            cell_mesh = filter_taubin(cell_mesh, iterations=50)

            ax.plot_trisurf(
                cell_mesh.vertices[:, 0],
                cell_mesh.vertices[:, 1],
                triangles=cell_mesh.faces,
                Z=cell_mesh.vertices[:, 2],
                lightsource=ls,
                alpha=1.0,
                color=cmap_gray_red(norm(measuremnt_lum.iloc[0])),
            )
        plt.axis("off")
        canvas = plt.gca().figure.canvas
        canvas.draw()
        data = np.frombuffer(canvas.tostring_rgb(), dtype=np.uint8)
        image = data.reshape(canvas.get_width_height()[::-1] + (3,))
        plt.close()
        # plt.imshow(image[100:450,150:500])
        # plt.show()
        skimage.io.imsave(
            f"updated_figures_figure_2/3D_colored/3D_lumen_{color_measurement}_{condition}_day_{int((t/24)+4)}.png",
            image[100:450, 150:500],
        )